Reinforcement Learning with OpenAI Gym
---
This notebook will create and test different reinforcement learning agents and environments.

In [2]:
import tensorflow as tf
import gym

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

Load the Environment
---
Call `gym.make("environment name")` to load a new environment.

Check out the list of available environments at <https://gym.openai.com/envs/>

Edit this cell to load different environments!

In [3]:
# TODO: Load an environment
env = gym.make("CartPole-v1")

In [4]:
# TODO: Print observation and action spaces
print(env.observation_space)
print(env.action_space)

Box(4,)
Discrete(2)


Run an Agent
---

Reset the environment before each run with `env.reset`

Step forward through the environment to get new observations and rewards over time with `env.step`

`env.step` takes a parameter for the action to take on this step and returns the following:
- Observations for this step
- Rewards earned this step
- "Done", a boolean value indicating if the game is finished
- Info - some debug information that some environments provide. 

In [4]:
# TODO Make a random agent
games_to_play = 10

for i in range(games_to_play):
    # Reset the environment
    obs = env.reset()
    episode_rewards = 0
    done = False
    
    while not done:
        # Render the environment so we can watch
        env.render()
        
        # Choose a random action
        action = env.action_space.sample()
        
        # Take a step in the environment with the chosen action
        obs, reward, done, info = env.step(action)
        episode_rewards += reward

    # Print episode total rewards when done
    print(episode_rewards)
    
# Close the environment
env.close()

14.0
9.0


26.0


16.0


12.0


33.0


36.0


15.0


27.0


13.0


Policy Gradients
---
The policy gradients algorithm records gameplay over a training period, then runs the results of the actions chosen through a neural network, making successful actions that resulted in a reward more likly, and unsuccessful actions less likely.

In [9]:
# TODO Build the policy gradient neural network
class Agent:
    def __init__(self, num_actions, state_size):
        
        initializer = tf.contrib.layers.xavier_initializer()
        
        self.input_layer = tf.placeholder(dtype=tf.float32, shape=[None, state_size])
        
        # Neural net starts here
        
        hidden_layer = tf.layers.dense(self.input_layer, 8, activation=tf.nn.relu, kernel_initializer=initializer)
        
        # Output of neural net
        self.outputs = tf.layers.dense(hidden_layer, num_actions, activation=tf.nn.softmax)
        self.choice = tf.argmax(self.outputs, 1)
        
        # Training Procedure
        self.rewards = tf.placeholder(shape=[None, 1], dtype=tf.float32)
        self.actions = tf.placeholder(shape=[None, 1], dtype=tf.int32)
        
        one_hot_actions = tf.one_hot(self.actions, num_actions)
    
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_actions, logits=self.outputs)
        
        self.loss = tf.reduce_mean(self.rewards * cross_entropy)
        
        self.tvars = tf.trainable_variables()
        
        self.gradients = tf.gradients(self.loss, self.tvars)
        
        # Compute gradients needed to make each action more likely
        #self.gradients = tf.gradients(loss, tf.trainable_variables())
        #self.adjusted_gradients = self.gradients * self.rewards
        
        # Create a placeholder list for gradients
        self.gradients_to_apply = []
        for index, variable in enumerate(tf.trainable_variables()):
            gradient_placeholder = tf.placeholder(tf.float32)
            self.gradients_to_apply.append(gradient_placeholder)
        
        # Create the operation to update gradients with the gradients placeholder.
        optimizer = tf.train.AdamOptimizer(learning_rate=1e-2)
        self.training_op = optimizer.minimize(self.loss)
        self.update_gradients = optimizer.apply_gradients(zip(self.gradients_to_apply, tf.trainable_variables()))
        

Discounting Rewards
---
In order to determine how "successful" a given action is, you need to track rewards over time in the environment. You could save each frame's reward at its full value, but in reality, actions are more likely to correlate to rewards closer to the current timeframe. "Discounting" rewards attempts to address this problem by scaling the value of a reward down the farther 

In [10]:
discount_rate = 0.99

def discount_normalize_rewards(rewards):
    discounted_rewards = np.zeros_like(rewards)
    total_rewards = 0
    
    for i in reversed(range(len(rewards))):
        total_rewards = total_rewards * discount_rate + rewards[i]
        discounted_rewards[i] = total_rewards
    
    discounted_rewards -= np.mean(discounted_rewards)
    discounted_rewards /= np.std(discounted_rewards)
    
    return discounted_rewards

Training Procedure
---
The agent will play games and record results. Every game, the gradients to apply will be calculated, and every few games they'll be averaged and applied.

In [11]:
tf.reset_default_graph()

agent = Agent(2,4)

training_episodes = 20000
max_steps_per_episode = 10000
episode_batch_size = 5

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    batch_history = []
    
    total_episode_rewards = []
    
    # Create a buffer of 0'd gradients
    gradient_buffer = sess.run(agent.tvars)
    for index, gradient in enumerate(gradient_buffer):
        gradient_buffer[index] = gradient * 0

    for episode in range(training_episodes):

        state = env.reset()
        
        episode_history = []
        episode_rewards = 0
        
        for step in range(max_steps_per_episode):
            
            if episode % 100 == 0:
                env.render()
            
            # Get weights for each action
            action_probabilities = sess.run(agent.outputs, feed_dict={agent.input_layer: [state]})
            if( episode % 100 == 0 and step % 10 == 0):
                print(action_probabilities[0])
            action_choice = np.random.choice(range(2), p=action_probabilities[0])
            
            state_next, reward, done, _ = env.step(action_choice)
            episode_history.append([state, action_choice, reward, state_next])
            state = state_next
            
            episode_rewards += reward
            
            if done:
                total_episode_rewards.append(episode_rewards)
                episode_history = np.array(episode_history)
                episode_history[:,2] = discount_normalize_rewards(episode_history[:,2])
                
                ep_gradients = sess.run(agent.gradients, feed_dict={agent.input_layer: np.vstack(episode_history[:, 0]),
                                                                    agent.actions: np.vstack(episode_history[:, 1]),
                                                                    agent.rewards: np.vstack(episode_history[:, 2])})
                # add the gradients to the grad buffer:
                for index, gradient in enumerate(ep_gradients):
                    gradient_buffer[index] += gradient
                    
                #Record episode events to the batch
                #batch_history.extend(episode_history)
                break
            
        if episode % episode_batch_size == 0:
            
            if episode % 100 == 0:
                print("Average reward / 100 eps: " + str(np.mean(total_episode_rewards[-100:])))
            # Run a training step with the batch we've collected
            #batch_history = np.array(batch_history)
            #batch_states = batch_history[:,0]
            #batch_actions = batch_history[:,1]
            #batch_rewards = batch_history[:,2]
            
            feed_dict_gradients = dict(zip(agent.gradients_to_apply, gradient_buffer))
            
            sess.run(agent.update_gradients, feed_dict=feed_dict_gradients)
            
            for index, gradient in enumerate(gradient_buffer):
                gradient_buffer[index] = gradient * 0
            
            batch_history = []
            
        

[ 0.49376717  0.5062328 ]
[ 0.40159589  0.59840411]


[ 0.42864743  0.5713526 ]
[ 0.47861791  0.52138209]


[ 0.42404425  0.57595575]
[ 0.49186847  0.5081315 ]


Average reward / 100 eps: 57.0


[ 0.48161393  0.51838613]
[ 0.49135968  0.50864023]


...

[ 0.99645209  0.00354794]
Average reward / 100 eps: 9.22


[ 0.99503243  0.00496752]
Average reward / 100 eps: 9.3


[ 0.99571061  0.00428933]
Average reward / 100 eps: 9.43


[ 0.99612021  0.00387985]
Average reward / 100 eps: 9.31


[ 0.99587387  0.00412612]
[  1.00000000e+00   1.64897082e-10]
Average reward / 100 eps: 9.41


In [38]:
env.close()